In [ ]:
#default_exp core

# Core

> Handle FHIR server communication and low-level resource functions.

In [ ]:
#export
import requests
from datetime import datetime, timezone

In [ ]:
import json

In [ ]:
#export
def get_bundle_as_raw_json(api_base, resource_type=None, url_suffix=None):
    "GET a bundle of resources of a specific type in JSON format"
    url=f'{api_base}/{resource_type}'
    if url_suffix is not None: 
        url+=url_suffix
    url+='&' if url_suffix is not None and '?' in url_suffix else '?'
    url+='_format=json'
    print('GET',url)
    return requests.get(url).json()

- `api_base` a "real" API base like `http://hapi.fhir.org/baseR4` 
- `resource_type` the type of resource to get. e.g. `Medication`
- `url_suffix` the preferred way to add search criteria. e.g. `?subject=d28f9c95-8098-4794-b1d0-57e45faf2b39`

To search for some medications;
```
get_bundle_as_raw_json('https://server.fire.ly/r4', 'Medication')
```

To get a single medication;
```
get_bundle_as_raw_json('https://server.fire.ly/r4', Medication', '?_id=d28f9c95-8098-4794-b1d0-57e45faf2b39')
```

In [ ]:
#export
def get_next_bundle_as_raw_json(json_response):
    "GET the next set of results"
    for link in json_response['link']:
        if link['relation'] == 'next':
            url = link['url']
            print('GET',url)
            return requests.get(url).json()

In [ ]:
#export
def timestamp_now():
    return datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')

In [ ]:
#export
def new_bundle(bundle_type='collection'):
    return dict(resourceType='Bundle', 
                type=bundle_type, 
                timestamp=timestamp_now(),
                entry=[])

In [ ]:
# this will fail if the bundle we create cannot be converted to JSON
json.dumps(new_bundle())

'{"resourceType": "Bundle", "type": "collection", "timestamp": "2021-09-14T08:52:48Z", "entry": []}'

In [ ]:
# Set the base URL of the FHIR server that we will use in tests
# api_base, resource_type = 'http://hapi.fhir.org/baseR4', 'ResearchStudy'
api_base = 'https://r4.smarthealthit.org'
patient_id = '11f2b925-43b2-45e4-ac34-7811a9eb9c1b'
resource_type = 'MedicationRequest'

The following cell shows how `get_bundle_as_raw_json` and `get_next_bundle_as_raw_json` can be used to read paged search results.

In [ ]:
bundle = get_bundle_as_raw_json(api_base, resource_type)
print(json.dumps(bundle, indent=2)[:500], '...')

GET https://r4.smarthealthit.org/MedicationRequest?_format=json
{
  "resourceType": "Bundle",
  "id": "bd829da7-f2c8-4974-a756-3a03dbf4f872",
  "meta": {
    "lastUpdated": "2021-09-14T04:52:49.045-04:00"
  },
  "type": "searchset",
  "link": [
    {
      "relation": "self",
      "url": "https://r4.smarthealthit.org/MedicationRequest?_format=json"
    },
    {
      "relation": "next",
      "url": "https://r4.smarthealthit.org?_getpages=bd829da7-f2c8-4974-a756-3a03dbf4f872&_getpagesoffset=50&_count=50&_format=json&_pretty=true&_bundletype=searchset"
    } ...


In [ ]:
json_response = get_bundle_as_raw_json(api_base, resource_type)
page_count = 1
while json_response is not None:    
    for i, entry in enumerate(json_response['entry']):
        if i > 2: break # show just 3 resources per "page"
        resource = entry.get('resource', {})
        print(f'{resource_type}:id', resource.get('id', 'missing'), resource.get('title', '')[:60])
    if page_count > 2: break # pull 3 pages at most to make testing fast
    json_response = get_next_bundle_as_raw_json(json_response)
    page_count += 1

GET https://r4.smarthealthit.org/MedicationRequest?_format=json
MedicationRequest:id f2947356-7ec6-4d36-bca6-0a38ccb0ffe5 
MedicationRequest:id 61ceaec7-6272-4975-81c3-ce2f216e6a38 
MedicationRequest:id 5180e54d-0187-494a-9ca9-9dbfcab90ab6 
GET https://r4.smarthealthit.org?_getpages=bd829da7-f2c8-4974-a756-3a03dbf4f872&_getpagesoffset=50&_count=50&_format=json&_pretty=true&_bundletype=searchset
MedicationRequest:id 6d91439c-0412-42a4-89e9-d50b7461add4 
MedicationRequest:id b94ee11a-57df-4510-b747-08cce91dfc8f 
MedicationRequest:id 023a4cce-acc8-4747-99f8-cf078a83c39b 
GET https://r4.smarthealthit.org?_getpages=bd829da7-f2c8-4974-a756-3a03dbf4f872&_getpagesoffset=100&_count=50&_format=json&_pretty=true&_bundletype=searchset
MedicationRequest:id ad518423-ff72-449d-8543-24b67f24dd11 
MedicationRequest:id c5f477df-9958-4aeb-9e50-99652dac841d 
MedicationRequest:id aed487ad-6960-46f0-b8c5-be115a2d73c4 


In [ ]:
#export
def extract_references_from_resource(resource, field_name):
    "Return a list of references extracted from a single resource and field"
    result = []
    if field_name in resource:
        references = resource[field_name]
        if not isinstance(references, list): references = [references]
        for reference in references:
            _reference = reference.get('reference')
            if _reference is None: continue
            if _reference.startswith('#'): continue
            # TODO: check that we have a relative reference or handle other kinds too
            result.append(_reference)
    return result

TODO: &uarr; we don't need to fetch contained references but we might want to make them bundle entries like the references that we do have to GET - check with Jay if this is already taken care of in FHIR to CDISC

In [ ]:
with open('test/patient_medication_bundle_0c4a1143-8d1c-42ed-b509-eac97d77c9b2.json') as f:
    test_bundle = json.load(f)
test_entry = test_bundle['entry'][3]
test_resource = test_entry['resource'] # resource with medicationReference

In [ ]:
assert (['Medication/bac1387e-3655-4e03-982f-7210faa21ea8'] 
        == extract_references_from_resource(test_resource, 'medicationReference'))

In [ ]:
#export
def extract_references(bundle, field_names):
    "Return a list of relative references e.g. `['Condition/1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8']`"
    if 'entry' not in bundle: return []
    result = []
    for entry in bundle['entry']:
        resource = entry.get('resource', {})
        for f in field_names:
            result.extend(extract_references_from_resource(resource, f))
    return list(set(result)) # de-duplicate but still return a list

In [ ]:
assert (['Condition/1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8', 'Medication/bac1387e-3655-4e03-982f-7210faa21ea8']
        == sorted(extract_references(test_bundle, ['medicationReference', 'reasonReference'])))

In [ ]:
# show that references are de-duplicated
test_bundle = new_bundle()
test_bundle['entry'].extend([test_entry, test_entry]) # create a bundle with duplicate medication references
assert (['Medication/bac1387e-3655-4e03-982f-7210faa21ea8']
        == extract_references(test_bundle, ['medicationReference', 'reasonReference']))

TODO: We need some config around how many and what type of issues clients can accept 
- some might want to fail if any piece of info cannot be found - i.e. references can't be followed
- some might want to get whatever is available and deal with the inconsistencies

In [ ]:
#export
def get_by_reference(api_base, reference):
    "Return a resource read from a FHIR server by reference, as a list containg a single bundle entry"
    if reference.startswith(api_base):
        reference = reference[len(api_base):].strip('/')
    if reference.startswith('http'):
        print(f'WARNING: Found reference {reference} that does not start with {api_base}')
        return []
    resource_type, id = reference.split('/')
    single_resource_bundle = get_bundle_as_raw_json(api_base, resource_type, f'?_id={id}')
    total = single_resource_bundle['total']
    if total != 1:
        print(f'WARNING: Expected a single resource for {reference} but found {total}')
        return []
#         raise Exception(f'Expected a single resource for {reference} but found {total}')
#         # TODO: we might want to print a warning and return an empyt list if this is too strict
    return single_resource_bundle['entry']

If a resource has a reference like
```
'medicationReference': {'reference': 'Medication/bac1387e-3655-4e03-982f-7210faa21ea8'},
```
we can HTTP GET the referenced resource (a `Medication` in this case) with http://hapi.fhir.org/baseR4/Medication?_id=bac1387e-3655-4e03-982f-7210faa21ea8

No problem if the reference includes the API base. This &darr; works the same as &uarr;
```
'medicationReference': {'reference': 'http://hapi.fhir.org/baseR4/Medication/bac1387e-3655-4e03-982f-7210faa21ea8'},
```

In [ ]:
if api_base == 'http://hapi.fhir.org/baseR4':
    test_id = 'bac1387e-3655-4e03-982f-7210faa21ea8'
    assert get_by_reference(api_base, f'Medication/{test_id}')[0]['resource']['id'] == test_id